In [18]:
import numpy as np
import torch
import torch.nn as nn
import statistics
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms, datasets
from torch.utils.data import DataLoader

print(torch.__version__)


2.9.1+cu128


In [11]:
# Question 1
criterion = nn.BCEWithLogitsLoss()

In [25]:
# Question 2
total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {total_params}")

Total parameters: 85249


In [16]:
# reproducibility 
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)


# Model 
class CNNBinaryClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        # Use padding=1 to preserve spatial dims before pooling
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)  
        self.bn1 = nn.BatchNorm2d(32)
        self.pool = nn.MaxPool2d(2, 2)                          

        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1) 
        self.bn2 = nn.BatchNorm2d(64)
        

        self.adaptive_pool = nn.AdaptiveAvgPool2d((4, 4))  

        self.fc1 = nn.Linear(64 * 4 * 4, 64)
        self.dropout = nn.Dropout(p=0.4)   
        self.fc2 = nn.Linear(64, 1)        

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.pool(x)

        x = F.relu(self.bn2(self.conv2(x)))
        x = self.pool(x)

        x = self.adaptive_pool(x)
        x = torch.flatten(x, 1)

        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        logits = self.fc2(x)
        return logits  


# instantiate model once and move to device
model = CNNBinaryClassifier().to(device)
print(model)

# criterion and optimizer 
criterion = nn.BCEWithLogitsLoss()  # stable: combines sigmoid+BCEloss internally
optimizer = optim.SGD(model.parameters(), lr=0.002, momentum=0.8)

# Data transforms 
train_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])])
val_transforms = train_transforms

# dataset paths 
train_dir = 'data/train'
test_dir  = 'data/test'

train_dataset = datasets.ImageFolder(root=train_dir, transform=train_transforms)
validation_dataset = datasets.ImageFolder(root=test_dir, transform=val_transforms)

train_loader = DataLoader(train_dataset, batch_size=20, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=20, shuffle=False)


# training loop 
num_epochs = 10
history = {'acc': [], 'loss': [], 'val_acc': [], 'val_loss': []}

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.float().unsqueeze(1).to(device)  

        optimizer.zero_grad()
        outputs = model(images)           
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)

        # compute predicted labels from logits
        probs = torch.sigmoid(outputs)
        predicted = (probs > 0.5).float()
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = correct_train / total_train
    history['loss'].append(epoch_loss)
    history['acc'].append(epoch_acc)

# validation
    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in validation_loader:
            images = images.to(device)
            labels = labels.float().unsqueeze(1).to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_running_loss += loss.item() * images.size(0)

            probs = torch.sigmoid(outputs)
            predicted = (probs > 0.5).float()
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_epoch_loss = val_running_loss / len(validation_dataset)
    val_epoch_acc = correct_val / total_val
    history['val_loss'].append(val_epoch_loss)
    history['val_acc'].append(val_epoch_acc)

    print(f"Epoch {epoch+1}/{num_epochs}  Loss: {epoch_loss:.4f}  Acc: {epoch_acc:.4f}  Val Loss: {val_epoch_loss:.4f}  Val Acc: {val_epoch_acc:.4f}")

# Save After training  :
torch.save(model.state_dict(), "cnn_binary_fixed.pth")



Device: cpu
CNNBinaryClassifier(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (adaptive_pool): AdaptiveAvgPool2d(output_size=(4, 4))
  (fc1): Linear(in_features=1024, out_features=64, bias=True)
  (dropout): Dropout(p=0.4, inplace=False)
  (fc2): Linear(in_features=64, out_features=1, bias=True)
)
Epoch 1/10  Loss: 0.6700  Acc: 0.5575  Val Loss: 0.6561  Val Acc: 0.7015
Epoch 2/10  Loss: 0.6150  Acc: 0.7100  Val Loss: 0.6096  Val Acc: 0.7065
Epoch 3/10  Loss: 0.5633  Acc: 0.7475  Val Loss: 0.5573  Val Acc: 0.7313
Epoch 4/10  Loss: 0.5179  Acc: 0.7712  Val Loss: 0.5532  Val Acc: 0.7711
Epoch 5/10  Loss: 0.4982  Acc: 0.782

In [19]:
# Question 3
median_acc = statistics.median(history['acc'])
print("Median training accuracy:", median_acc)


Median training accuracy: 0.788125


In [20]:
# Question 4
losses = [0.6586,0.5467,0.5127,0.4621,0.4185,0.3805,0.2969,0.2143,0.2472,0.1975]
print("sample std dev:", statistics.stdev(losses))
print("population std dev:", statistics.pstdev(losses))

sample std dev: 0.15445989770811064
population std dev: 0.1465335251742754


In [21]:
# Augmented train transforms (as requested)
aug_train_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.RandomRotation(50),
    transforms.RandomResizedCrop(200, scale=(0.9,1.0), ratio=(0.9,1.1)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])])

# Replace the train dataset and loader 
train_dataset.transform = aug_train_transforms
train_loader = DataLoader(train_dataset, batch_size=20, shuffle=True)


# Continue training for 10 more epochs
num_more_epochs = 10

for epoch in range(num_more_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.float().unsqueeze(1).to(device)

        optimizer.zero_grad()
        outputs = model(images)  # logits
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        probs = torch.sigmoid(outputs)
        predicted = (probs > 0.5).float()
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = correct_train / total_train
    history['loss'].append(epoch_loss)
    history['acc'].append(epoch_acc)

    # validation (test) step
    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in validation_loader:
            images = images.to(device)
            labels = labels.float().unsqueeze(1).to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_running_loss += loss.item() * images.size(0)
            probs = torch.sigmoid(outputs)
            predicted = (probs > 0.5).float()
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_epoch_loss = val_running_loss / len(validation_dataset)
    val_epoch_acc = correct_val / total_val
    history['val_loss'].append(val_epoch_loss)
    history['val_acc'].append(val_epoch_acc)

    print(f"[AUG] Epoch {epoch+1}/{num_more_epochs}  Loss: {epoch_loss:.4f}  Acc: {epoch_acc:.4f}  Val Loss: {val_epoch_loss:.4f}  Val Acc: {val_epoch_acc:.4f}")




[AUG] Epoch 1/10  Loss: 0.5090  Acc: 0.7775  Val Loss: 0.4730  Val Acc: 0.7910
[AUG] Epoch 2/10  Loss: 0.4949  Acc: 0.7712  Val Loss: 0.4491  Val Acc: 0.7960
[AUG] Epoch 3/10  Loss: 0.4709  Acc: 0.7875  Val Loss: 0.4618  Val Acc: 0.7662
[AUG] Epoch 4/10  Loss: 0.4847  Acc: 0.7887  Val Loss: 0.4521  Val Acc: 0.7662
[AUG] Epoch 5/10  Loss: 0.4749  Acc: 0.7662  Val Loss: 0.4661  Val Acc: 0.7861
[AUG] Epoch 6/10  Loss: 0.4716  Acc: 0.7900  Val Loss: 0.4485  Val Acc: 0.7711
[AUG] Epoch 7/10  Loss: 0.4544  Acc: 0.7875  Val Loss: 0.5135  Val Acc: 0.7512
[AUG] Epoch 8/10  Loss: 0.4707  Acc: 0.7987  Val Loss: 0.5112  Val Acc: 0.7512
[AUG] Epoch 9/10  Loss: 0.4580  Acc: 0.8037  Val Loss: 0.4339  Val Acc: 0.7861
[AUG] Epoch 10/10  Loss: 0.4465  Acc: 0.8037  Val Loss: 0.4832  Val Acc: 0.7612


In [23]:
# After training, metrics for Question 5
# Mean of test loss for all epochs
mean_test_loss_all_epochs = float(np.mean(history['val_loss']))
print("Mean test (val) loss over all recorded epochs:", mean_test_loss_all_epochs)


Mean test (val) loss over all recorded epochs: 0.5112702000422857


In [24]:
# Q6: average of test accuracy for the last 5 epochs. 
aug_block_val_acc = history['val_acc'][-10:]   # last 10 epochs (augmentation block)
avg_last5_aug = float(np.mean(aug_block_val_acc[-5:]))
print("Average val accuracy for last 5 epochs of augmentation block:", avg_last5_aug)


Average val accuracy for last 5 epochs of augmentation block: 0.7641791044776121
